In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [8]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [9]:
df = pd.read_csv('../datasets/car.csv')

In [10]:
df.shape

(1728, 7)

In [11]:
bdf = BaseDatasetTransform(df,target='Car_Acceptability')

In [12]:
X_train, X_test, y_train, y_test,categorical_features = bdf.fit_transform()

Series([], dtype: int64)

В колонке(ах) ['Car_Acceptability'] нет пропущенных значений
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['Buying_Price', 'Maintenance_Price', 'No_of_Doors', 'Person_Capacity',
       'Size_of_Luggage', 'Safety'],
      dtype='object')

Числовые колонки:
Index([], dtype='object')


In [14]:
lgbm_model = lgb.LGBMClassifier(verbose=-1)

columns_to_delete = []  

unique_classes = len(set(y_train))

model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    objective='multiclass',  
    num_class=unique_classes, 
    metric='multi_error' 
)

In [15]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


ModelEvaluator(metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=['Buying_Price',
                                                                  'Maintenance_Price',
                                                                  'No_of_Doors',
                                                                  'Person_Capacity',
                                                                  'Size_of_Luggage',
                                                                  'Safety'],
                                            columns_to_delete=[],
                                            model=LGBMClassifier(metric='multi_error',
                                                                 num_class=1,
                                                                 objective='multiclass',
                                                                 verbose=-1)))

In [16]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,Metric,Score
0,accuracy,0.990366
1,precision,0.989583
2,recall,0.996556
3,f1,0.992947
